In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Example data: 5 samples, each with 2 features
X_train = torch.tensor([
    [0.1, 0.2],
    [0.4, 0.2],
    [0.3, 0.4],
    [0.6, 0.7],
    [0.2, 0.9]
], dtype=torch.float32)

# Example soft labels: 5 samples, each with a probability distribution over 3 classes
y_train_prob = torch.tensor([
    [0.2, 0.5, 0.3],
    [0.1, 0.7, 0.2],
    [0.4, 0.4, 0.2],
    [0.3, 0.3, 0.4],
    [0.6, 0.1, 0.3]
], dtype=torch.float32)

# Define a simple neural network model
class SoftLabelNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SoftLabelNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
hidden_size = 10
output_size = y_train_prob.shape[1]

model = SoftLabelNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert soft labels to one-hot encoding for the CrossEntropyLoss
# (PyTorch expects class indices for CrossEntropyLoss, not probabilities)
y_train_indices = torch.argmax(y_train_prob, dim=1)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train_indices)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Example test data
X_test = torch.tensor([
    [0.2, 0.1],
    [0.3, 0.6]
], dtype=torch.float32)

# Predict the probability distribution over classes for new samples
with torch.no_grad():
    predicted_probabilities = model(X_test)
    print("Predicted Probabilities:")
    print(predicted_probabilities)


Epoch [50/500], Loss: 0.9297
Epoch [100/500], Loss: 0.8397
Epoch [150/500], Loss: 0.7909
Epoch [200/500], Loss: 0.7179
Epoch [250/500], Loss: 0.5997
Epoch [300/500], Loss: 0.5725
Epoch [350/500], Loss: 0.5637
Epoch [400/500], Loss: 0.5597
Epoch [450/500], Loss: 0.5575
Epoch [500/500], Loss: 0.5561
Predicted Probabilities:
tensor([[4.5012e-05, 9.9995e-01, 2.0830e-08],
        [9.8795e-01, 1.1566e-05, 1.2036e-02]])
